# Contexto

O objetivo deste notebook é demonstrar algumas técnicas de extração de informação utilizando videos de melhores momentos de partidas de futebol (*). Com o córpus, iremos tentar primeiramente retirar informações importantes como placar, melhores jogadores, resumo curto, resumo longo e com estes dados e uma função de resgate de contexto (RAG) construir um sistema que possa ser usado para um simples **sistema de perguntas e respostas**, como perguntas como:

- Quanto foi o jogo entre XXXX e YYYY?
- Quem fez os gols do time XXXX?
- Alguém foi expulso na partida?

Vale ressaltar que as respostas das perguntas devem estar na transcrição do vídeo para que o modelo consiga recuperar este contexto.

\(*) _Em um projeto mais extenso, seria ideal que o córpus do projeto fosse transcrições inteiras de jogos, e que estas transcrições fossem feitas por um modelo com mais acurácia que o de transcrição automática do YouTube porém, dado os prazos e limites da matéria, foram utilizados a transcrição automática e vídeos de melhores momentos._


## Integrantes

**11202020600 FERDINANDO LONGONI**

**11202111177 LUCAS GOIS**

**11202131332 RENAN SANTANA**

# Técnicas utilizadas:
- Extração de informação
- Sumarização
- Tokenização / Vetorização
- Embeddings
- Sistema de perguntas e respostas

## Grande modelo de linguagem (*Large Language Model - LLM*)

---

**LLM**: 
 - Chat: Gemini (gemini-2.5-flash-lite)
 - Embedding: Gemini (gemini-embedding-001)

**Link para a documentação oficial**: https://ai.google.dev/gemini-api/docs?hl=pt-br



## Obtenção do córpus

Para obter o córpus, iremos utilziar a biblioteca `pytubefix` que é uma interface para a API do Youtube em python.

In [2]:
#Setup

from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

TEXT_DOWNLOAD_PATH  = "transcriptions"

os.makedirs(TEXT_DOWNLOAD_PATH, exist_ok=True)

In [ ]:
token= os.getenv("GEMINI_API_KEY")

# url = "https://www.youtube.com/watch?v=7D4Tfab_X4k"
# url = "https://www.youtube.com/watch?v=kjAg7CWhiDU"
url = "https://www.youtube.com/watch?v=tZloT4HKXmk"

yt = YouTube(url, on_progress_callback=on_progress)
print(f"O título do vídeo selecionado é: {yt.title}\nDuração de: {yt.length}s")


O título do vídeo selecionado é: BARCELONA FAZ NO FIM DA PRORROGAÇÃO, vence jogaço contra o REAL MADRID e é CAMPEÃO da Copa do Rei
Duração de: 1103s


Além disso, temos que verificar se o vídeo selecionado possui transcrição e em qual idioma está.

In [4]:
captions = yt.captions

print("Idiomas disponíveis: ")

for idiom_code in captions:
    print(f"Code: {idiom_code}")
    print(f"Name: {idiom_code.name}")

if captions is None:
    print("O vídeo selecionado não possui transcrição! Favor selecionar outro.")

Idiomas disponíveis: 
Code: <Caption lang="Portuguese" code="pt">
Name: Portuguese


In [12]:
# Obter a transcrição
idiom_code = "pt"
transcription = captions.get_by_language_code(idiom_code)

# O formato da transcrição por padrão é SRT, que contém junto com a transcrição os timestamps de cada frase. 
# Entretanto, para a nossa aplicação, este dado é inútil e portanto iremos utilizar apenas o texto.

text_transcription = transcription.generate_txt_captions()

with open(f"{TEXT_DOWNLOAD_PATH}/{yt.title.replace(" ", "_")}.txt", "w") as f:
    f.write(text_transcription)


/tmp/ipykernel_768694/2477874501.py:3: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  transcription = captions.get_by_language_code(idiom_code)


Com estes procedimentos feitos, o córpus está pronto (em `text_transcription`) e podemos alimentar o modelo com a transcriçao.

## Sumarização, Embedding e Chat

Com o córpus pronto, iremos começar os procedimentos relacionados a LLM, utilizando a biblioteca LangChain como auxiliadora neste processo.

Inicializaremos o modelo usado para o **chat**.

In [6]:
import os
from langchain.chat_models import init_chat_model

llm = init_chat_model("google_genai:gemini-2.5-flash")

# Outra maneira de inicalizar uma LLM, utilizando o GitHub Models

# import os
# from langchain_openai import ChatOpenAI

# token = os.environ["GITHUB_TOKEN"]
# endpoint = "https://models.github.ai/inference"
# model = "openai/gpt-4.1"

# llm = ChatOpenAI(
#     api_key=token,
#     base_url=endpoint,
#     model=model
# )


Tranformação da transcrição em um DocumentLoader do LangChain e a separação do texto em chunks.

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

TEXT_FILE = f"trabalho/{TEXT_DOWNLOAD_PATH}/ldu.txt"
TEXT_FILE = "/home/ferdinando-longoni/ufabc/PLN.2025.3/trabalho/transcriptions/ldu.txt"
TEXT_FILE = f"transcriptions/{yt.title.replace(" ", "_")}.txt"
CHUNK_SIZE = 200
CHUNK_OVERLAP = 10

loader = TextLoader(TEXT_FILE)

transcription = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,  # chunk size (characters)
    chunk_overlap=CHUNK_OVERLAP,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

all_splits = text_splitter.split_documents(transcription)

print(f"A transcrição foi separada em {len(all_splits)} sub-documentos.")

A transcrição foi separada em 63 sub-documentos.


### Sumarização

Iremos criar a estrutura da resposta que queremos, e posteriormente usar esta estrutura para o sistema de perguntas e respostas (juntamente com a busca de contexto).

Para a sumarização, por conta do pequeno córpus (alguns milhares de tokens) utilizaremos a _Direct Summarization_. Caso fosse utilizada a tanscrição completa, seria necessária a utilização de algum método mais delicado como _refine_, _map-reduce_ ou _stuff_.

In [14]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Jogador(BaseModel):
    nome: str
    time: str

    def __str__(self):
        return f"{self.nome} ({self.time})"
    
class CountJogadas(BaseModel):
    jogador: Jogador
    count: int

class Goal(BaseModel):
    jogador: Jogador
    ordem: int
    time: str
    assistencia: Optional[Jogador]

class Game(BaseModel):
    score: str = Field(description="O placar final do jogo no formato TIME 1 Y x Z TIME 2")
    goals: List[Goal] = Field(description="Lista de gols. Não invente dados caso não saiba.")
    best_players: List[Jogador] = Field(description="Lista com exatamente 3 jogadores foram melhores na partida'")
    short_summary: str = Field(description="Resumo do jogo, em até 200 caracteres.")
    long_summary: str = Field(description="Resumo o mais detalhado possível do jogo citando cada jogada o mais minuciosamente possível.")
    list_of_players: List[CountJogadas] = Field(description="O número de vezes que cada jogador participou de alguma jogada importante.")


Juntaremos todos esses procedimentos e iremos passar para a LLM selecionada, junto com o prompt.

In [15]:
model_with_struct = llm.with_structured_output(Game)

prompt = f"""
Você é um extrator altamente preciso.
                                    
Sua tarefa é analisar o texto fornecido e preencher corretamente todos os campos do objeto `Game` que será retornado pela LLM através de structured output.

Siga estas regras:

1. É possível conjecturar dados se forem muito implicitos (por exemplo, uma equipe nao marcou gols, placar do time será 0)
2. O texto pode conter erros gramaticais, então caso precise voce pode inferir que dois nomes semelhantes são o mesmo jogador..
3. Para `goals`, identifique cada gol citado, sua ordem e quem marcou. Se não houver informação sobre assistência, deixe como null.
4. `best_players` deve ter exatamente 3 jogadores. Caso o texto não esclareça, escolha os mais envolvidos em jogadas relevantes.
5. `list_of_players` deve conter todos os jogadores mencionados e o número de participações importantes (passes decisivos, finalizações, roubadas de bola, gols, assistências, defesas).
6. `short_summary` deve ter no máximo 200 caracteres.
7. `long_summary` deve ser detalhado, mencionando cada jogada importante citada e em formato de prosa.
8. Se algum campo não puder ser inferido, deixe vazio (lista vazia) ou null quando permitido.
9. O modelo retornará diretamente um objeto do tipo Game, então não inclua JSON ou texto fora dos valores dos campos.

                                    
Texto:
{transcription[0].page_content}
"""

result = model_with_struct.invoke(prompt)

#### Resultados:

In [16]:
# Bruto, em formato json:

print(f"{result.model_dump_json(indent=2, ensure_ascii=False)}")

{
  "score": "Barcelona 3 x 2 Real Madrid",
  "goals": [
    {
      "jogador": {
        "nome": "Pedri",
        "time": "Barcelona"
      },
      "ordem": 1,
      "time": "Barcelona",
      "assistencia": {
        "nome": "Lamine Yamal",
        "time": "Barcelona"
      }
    },
    {
      "jogador": {
        "nome": "Kylian Mbappé",
        "time": "Real Madrid"
      },
      "ordem": 2,
      "time": "Real Madrid",
      "assistencia": null
    },
    {
      "jogador": {
        "nome": "Tchouaméni",
        "time": "Real Madrid"
      },
      "ordem": 3,
      "time": "Real Madrid",
      "assistencia": {
        "nome": "Arda Guler",
        "time": "Real Madrid"
      }
    },
    {
      "jogador": {
        "nome": "Ferran Torres",
        "time": "Barcelona"
      },
      "ordem": 4,
      "time": "Barcelona",
      "assistencia": {
        "nome": "Lamine Yamal",
        "time": "Barcelona"
      }
    },
    {
      "jogador": {
        "nome": "Jules Koundé",
  

Agora, inicializaremos o modelo utilizado para o embedding, ou seja, o modelo que será utilizado para vetorização das partes do nosso córpus;

Resumos em um print mais organizado:

In [17]:
from pprint import pprint

pprint(f"Resumo curto: {result.short_summary}", width=80)
pprint(f"Resumo longo: {result.long_summary}", width=80)

('Resumo curto: O Barcelona venceu o Real Madrid por 3 a 2 na final da Copa do '
 'Rei 2024/2025 na prorrogação. Pedri abriu o placar, Mbappé e Tchouaméni '
 'viraram para o Madrid, Ferran Torres empatou e Koundé marcou o gol da '
 'vitória.')
('Resumo longo: A final da Copa do Rei 2025 entre Real Madrid e Barcelona em '
 'Lacarturra começou com grande expectativa. O Barcelona, sem Lewandowski e '
 'Balde, buscou um jogo de posse de bola e ataques rápidos de Lamine Yamal e '
 'Rafinha. O Real Madrid focou em jogadas individuais. A primeira chance '
 'perigosa foi de Lamine Yamal, que cortou para o meio e finalizou com perigo. '
 'Um escanteio de Rafinha encontrou Koundé, cujo cabeceio foi defendido por '
 'Courtois. Vinicius Júnior criou uma chance, mas Cubarci fez uma leitura '
 'espetacular, lançando Lamine Yamal contra Fran Garcia. Lamine Yamal então '
 'passou para Pedri, que, aos 28 minutos do primeiro tempo, marcou um belo gol '
 'de primeira no ângulo, abrindo o placar em 1 a 0 

In [18]:
print(f"Placar: {result.score}")
print(f"Melhores jogadores: {result.best_players}")
print(f"Participacoes dos jogadores: {result.list_of_players}")

Placar: Barcelona 3 x 2 Real Madrid
Melhores jogadores: [Jogador(nome='Lamine Yamal', time='Barcelona'), Jogador(nome='Pedri', time='Barcelona'), Jogador(nome='Jules Koundé', time='Barcelona')]
Participacoes dos jogadores: [CountJogadas(jogador=Jogador(nome='Lamine Yamal', time='Barcelona'), count=9), CountJogadas(jogador=Jogador(nome='Pedri', time='Barcelona'), count=5), CountJogadas(jogador=Jogador(nome='Jules Koundé', time='Barcelona'), count=6), CountJogadas(jogador=Jogador(nome='Kylian Mbappé', time='Real Madrid'), count=7), CountJogadas(jogador=Jogador(nome='Vinicius Júnior', time='Real Madrid'), count=6), CountJogadas(jogador=Jogador(nome='Ter Stegen', time='Barcelona'), count=8), CountJogadas(jogador=Jogador(nome='Fran Garcia', time='Real Madrid'), count=5), CountJogadas(jogador=Jogador(nome='Courtois', time='Real Madrid'), count=5), CountJogadas(jogador=Jogador(nome='Jude Bellingham', time='Real Madrid'), count=4), CountJogadas(jogador=Jogador(nome='Rafinha', time='Barcelona')

### Embedding

Com o sumário e outras informações resgatadas, iremos começar com embedding do córpus.

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# from langchain_cohere import CohereEmbeddings

# embeddings = CohereEmbeddings(model="embed-v4.0")


Inicialização do vector store. 

Escolhemos o FAISS (Facebook AI Similarity Search).

In [20]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


Adição dos documentos ao vector store

In [21]:
document_ids = vector_store.add_documents(documents=all_splits)

Iremos usar o decorator `@tool` do langchain para a criação do RAG

In [23]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

### Chat

Com estes procedimentos feitos, configuraremos o modelo de chat, passando um prompt adequado que terá a informação do processo de sumarização e também
ao de resgate de contexto. (RAG)

In [24]:
from langchain_openai import ChatOpenAI, custom_tool
from langchain.agents import create_agent

prompt = (f"""
    Você é um auxiliar que deverá responder perguntas relacionadas a uma partida de futebol.Para
    isto você terá a transcrição de um vídeo dos melhores momentos da partida e um json estruturado
    com algumas informações já processadas.

    
    Dados:
    {result.model_dump_json()}   

    Transcrição:
    {transcription[0].page_content}
""")


agent = create_agent(llm, [retrieve_context], system_prompt=prompt)


In [ ]:
query = "Quem foi o melhor jogador da partida? E por que?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Quem foi o melhor jogador da partida? E por que?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'O melhor jogador da partida foi Lamine Yamal, do Barcelona. Ele foi fundamental na criação de jogadas de ataque, dando assistências para dois gols do Barcelona, o primeiro de Pedri e o de empate de Ferran Torres. Ele também teve uma chance perigosa no início do jogo.', 'extras': {'signature': 'CtwCAdHtim991Fy2nX0bb8vjmoveUKJ5PeoyS91sH/k8pNvxmMuT3Ne5Pl4RpAm6H3tBuJDtehY57Bje0i9C3QRE3wo43k8uY04k4IBTycuFOwxD8vzeHrQ52Jd00R4gNU0eUnKYE43ujW1wpBns7Mco1ojuRRKsh2CXJnXH4YoR2isYMBt07Up0tUiRN06VM6/vlMzQswrVBoDugf+Xwm+HeFFlvyjLph6AUKmTeokzQITRUJf/3YFLTmpJuzqEc5+/Pya0PtwLtJOvNjRx5NoHcZevLHlKoNQfZ+YmIKvqk41VUGRtrhOImpNXZZYl8i3vbdgVYjYkA/1z2ozlGOwnAyZAC9iIgD+PmS6g5anUBw2akKjIoj4duTkj3LeXjMPaXxhkaFF9UoAN3/uia6r325PO6EGnsN0FQbD1UQts+/AaF/kquBV5NOn/TfnUvaGiA6KT4srG/g+zv

In [28]:
query = "Alguem falhou no jogo?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Alguem falhou no jogo?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Sim, no segundo tempo da prorrogação, aos 10 minutos, houve um erro de passe de Modric, que permitiu a Jules Koundé interceptar a bola, avançar e marcar o gol da vitória para o Barcelona.', 'extras': {'signature': 'Cq0fAdHtim/eDUfwxK5K/pxpkY4T7X49hxW1c8uMTEjtPkAhU+wVoWIKTJYeIjIKZf/7fFV/wcd6qcQyiuo7xGDMsRi2KtDlqIaAzsQbjb6i9WQOwzR67kiDZJeRjE2wMseOzSGKujf+YX9vWHeMm3OjyBC0jItaIyZ9xvdYl/6uhP1t7oiqMjkjhMnw9XlvtQiyC2tiNK4VjkVHD8drtIrS55D3e6duuMguHmBj5vRpgEl8jZ7qjADdpKd/j81yWExEAU9HrpdSEniItZU5vA9PmNqMBfRJfk3cUbC3j5L3/4ykT3UXZs6tPZ6RGAzSS2s63S6TUcBWsGpW+STmbhKAdzQAnPOeHRRqrirCDHV97pipBaFTKlQGhR+jBs7JiMNM1ql2cK7VOOjq/XwCk8ypM/lYl0bE/SfyCGRlgJqw4mJxanWs3KFLviObBJtmm2QFXGQL/Obf8AWO1obkOet9Gy3wKkMPPQPdc/XB0frGRqknJtP2VZK+zWar4qs0AaxJ7KN/oV1Vl20IQ/rajpeZIjOJlittcdHxEnKQx9PIbIEL6aXphwDVoAm

In [29]:
query = "Aonde foi o jogo?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Aonde foi o jogo?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'O jogo foi em Lacarturra.', 'extras': {'signature': 'CqYDAdHtim8tNuVXZ4JrmyDR8WAQlFWA57RkaReFBHCrkmdTJZ9SrQBfhfQMm6bJWAAeEdD/zyuUjYPa70VA68O5D3ea8tFst6XUGHsCmlMd1keK03voGT0EMlQRY6V3wwSykp27sKVOc8nwJH2rS67Gzi2hBedjmgwc+CTWOeGwyCngnR8rubyPcFnyORqgzVXwiTFsxto5VeCfQOGZonZWHhX2GnJArTSoPn08uq2ds2QqlOc7G/yT/AbK98mUpbKAlT4j2BDXwqzyKSN1L6VjmWaX4NzfW90O59wVIzEPb2iJXCh0A+JjeL/5BdsfTNnZSEWo3y/qAlq7ygGI54r5DkqT2cTNWU0EBPH4HQSw9wB7aAvSjzmxnASH1/SliWs5iuOAkiPWEydwW0wLtq19wpjssTFpU+FK6EzOgn7X2axu9nj+OqCLyGM6XJRKsG5+Wi3tdd0BTUoADbyCkClgAGF8pY/xKdQfNuLqFS/8vv/6j8nvj9BsPxQot8R3gz3/ZFnobI3dOVE+vCIidIT/1C8xCq3iBHQbCiAXI4zF9gldT6JxxI0='}}]


In [31]:
query = "Vinicius junior jogou?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Vinicius junior jogou?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (6f9e2f1a-2ddb-4e4f-8f69-222f1bd20cf4)
 Call ID: 6f9e2f1a-2ddb-4e4f-8f69-222f1bd20cf4
  Args:
    query: Vinicius Junior played in the match
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'transcriptions/BARCELONA_FAZ_NO_FIM_DA_PRORROGAÇÃO,_vence_jogaço_contra_o_REAL_MADRID_e_é_CAMPEÃO_da_Copa_do_Rei.txt', 'start_index': 3598}
Content: Vinícius Júnior ele leva pé direito vem  a batida defendeu Chny aí do Vine Chesne de novo duas vezes o Vinícius Júnior buscando o  gol de empate chney fechando a porta tudo começa na recuperação de

Source: {'source': 'transcriptions/BARCELONA_FAZ_NO_FIM_DA_PRORROGAÇÃO,_vence_jogaço_contra_o_REAL_MADRID_e_é_CAMPEÃO_da_Copa_do_Rei.txt', 'start_index': 2847}
Content: 